In [ ]:
#| hide
from uom_project import (
    core, poisson_solvers, streamfunction_vorticity,
    streamfunction_vorticity_newton,
)

from functools import partial

import numpy as np

# uom-project

> Repository containing code and and other materials for my undergraduate project on numerical solutions to the Navier-Stokes equations at the University of Manchester.

## Install

```sh
pip install uom_project
```

## How to use

### Poisson solvers

In [ ]:
# Setup the problem
w, exact_solution = core.setup_poisson_problem(N=20)[:2]

#### Gauss-Seidel

In [ ]:
psi = poisson_solvers.poisson_gauss_seidel_with_sor_solver(w, verbose=False)[0]
np.abs(psi - exact_solution).max() # Check absolute error is small

0.0020587065214789924

#### SciPy solvers

In [ ]:
psi = poisson_solvers.poisson_non_iterative_solver(w, algorithm="base")
np.abs(psi - exact_solution).max() # Check absolute error is small

0.002058706764533902

In [ ]:
psi = poisson_solvers.poisson_non_iterative_solver(w, algorithm="bicgstab")
np.abs(psi - exact_solution).max() # Check absolute error is small

0.00205870676453368

#### Newton's and similar solvers

In [ ]:
psi = poisson_solvers.poisson_newton_solver(w)
np.abs(psi - exact_solution).max() # Check absolute error is small

0.002058706764533902

In [ ]:
options = { "line_search": None, "jac_options": { "reduction_method": "svd" } }
psi, solution = poisson_solvers.poisson_newton_alternative_solver(
    w, method="broyden2", options=options
)
np.abs(psi - exact_solution).max() # Check absolute error is small

0.002058706764533458

### Solving the full Navier-Stokes equations in the streamfunction-vorticity form

In [ ]:
N = 20
nx, ny, x_grid, y_grid = core.setup_poisson_problem(N=N)[2:]
h = 1 / N


U_wall_top = np.sin(np.pi * np.arange(1, nx - 1) * h) ** 2
Re = 10 # i.e. mu = 0.1

tfinal = 3.5
dt = 0.2 * Re / (N ** 2)
dpic = 0.5

w, psi, _ = streamfunction_vorticity.streamfunction_vorticity_iterative_solver(
    N=N, Re=Re, tfinal=tfinal, U_wall_top=U_wall_top, dt=dt, print_every=dpic
)

t=0.50000; w(0.5, 0.5)=-0.5345059709566309
t=1.00000; w(0.5, 0.5)=-0.639156573004994
t=1.50000; w(0.5, 0.5)=-0.6466476560907706
t=2.00000; w(0.5, 0.5)=-0.6471822817711572
t=2.50000; w(0.5, 0.5)=-0.6472204426025077
t=3.00000; w(0.5, 0.5)=-0.6472231665567474
t=3.50000; w(0.5, 0.5)=-0.6472233609961568


In [ ]:
w2, psi2, _ = streamfunction_vorticity_newton.newton_solver(
    f=partial(streamfunction_vorticity_newton.f, U_wall_top=U_wall_top),
    get_jacobian=streamfunction_vorticity_newton.get_jacobian,
    N=N, Re=Re, quiet=False,
)

np.allclose(w, w2, ), np.allclose(psi, psi2)

iter=1; residual=1.0836381666440038; dx=39.71450891546238
iter=2; residual=0.0015404929313134973; dx=2.2739268203013605
iter=3; residual=8.799679858681107e-10; dx=0.0014642597837430872
n_iter=3


(True, True)